# Object detection train with TensorFlow 2.4.1
This notebook will train a model for the object detection purpouse.
It can be run as a Jupiter notebook in the Google Colab environment or exported as a Python file and run from a command line.


#Model types
Initialize the list of the available pre-trained models and their parameters

In [ ]:
""" List of the available models and their definitions """
models = {
    'SSD MobileNet v2 320x320': {
        'DownloadPath': 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz',
        'batch_size': 12,
        'height': 300,
        'width': 300
    },
    'SSD ResNet50 V1 FPN 640x640 (RetinaNet50)': {
        'DownloadPath': 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz',
        'batch_size': 8,
        'height': 640,
        'width': 640
    },
}

if __name__ == '__main__':
    import pprint
    pprint.PrettyPrinter(1).pprint(models)
    print('Dictionary of pre-trained models configured')


#Base train parameters
**model_type:**

>The chosen base pre-trained model type (from the list in the previous cell). Write it between quotation marks.

**model_dir:**
> the output directry for the trained models' checkpoints.

**train_images_dir:**
> the directory on your Google Drive containing the images for the train and their xml annotations. You can use standard images annotation tools as [labelImg](https://github.com/tzutalin/labelImg), [tVoTT](https://github.com/microsoft/VoTT), etc... for the annotation task.
Put the xml generated files in te same directory of the images.

**eval_images_dir:**
> the directory on your Google Drive containing the images used for evaluating the train. They could be near from 10% to 20% of the number of the train images. They must be labeled as the train images.


In [ ]:
import  os
from    pathlib import Path
import  tempfile
import  sys

class BaseParameters:
    """ Class holding the base parameters """
    def __init__(self):
        """ Constructor """
        self._flags = flags.FLAGS
        self._flags.mark_as_parsed()
        self._model_type = 'SSD ResNet50 V1 FPN 640x640 (RetinaNet50)'
        self._model_dir = 'trained-model'
        self._train_images_dir = 'images/train'
        self._eval_images_dir = 'images/eval'
    @property
    def model_type(self): return self._flags.model_type or self._model_type
    @property
    def model_dir(self): return self._flags.model_dir or self._model_dir
    @model_dir.setter
    def model_dir(self, value): self._flags.model_dir = self._model_dir = value
    @property
    def train_images_dir(self): return self._flags.train_images_dir or self._train_images_dir
    @train_images_dir.setter
    def train_images_dir(self, value): self._flags.train_images_dir = self._train_images_dir = value
    @property
    def eval_images_dir(self): return self._flags.eval_images_dir or self._eval_images_dir
    @eval_images_dir.setter
    def eval_images_dir(self, value): self._flags.eval_images_dir = self._eval_images_dir = value
    @property
    def annotations_dir(self): return 'annotations'
    @property
    def model(self):
        global models
        return models[self.model_type]
    @property
    def pre_trained_model_base_dir(self): return os.path.join(tempfile.gettempdir(), "tensorflow-pre-trained-models")
    @property
    def pre_trained_model_dir(self):
        return str(Path(os.path.join(self.pre_trained_model_base_dir, Path(self.model["DownloadPath"]).name)).with_suffix("").with_suffix(""))

""" Arguments definition """
from absl import flags
if (not 'model_type' in flags.FLAGS):
    flags.DEFINE_string(
        'model_type',
        'SSD ResNet50 V1 FPN 640x640 (RetinaNet50)',
        'Type of the base model.')
    flags.DEFINE_string(
        'model_dir',
        'trained-model',
        'Path to output model directory where event and checkpoint files will be written.')
    flags.DEFINE_string(
        'train_images_dir',
        'images/train',
        'Path to the directory containing the images for train and their labeling xml.')
    flags.DEFINE_string(
        'eval_images_dir',
        'images/eval',
        'Path to the directory containing the images for evaluate and their labeling xml.')

prm = BaseParameters()
print(f'Model type: {prm.model_type}')
print(f'Output directory: {str(Path(prm.model_dir).resolve())}')
print(f'Train images directory: {str(Path(prm.train_images_dir).resolve())}')
print(f'Evaluation images directory: {str(Path(prm.eval_images_dir).resolve())}')
print('Base parameters configured')


#Train parameters
A list of train parameters. Read the comments in the flags section for the usage

In [ ]:
import  sys

class TrainParameters(BaseParameters):
    """ Class holding the train execution parameters """
    def __init__(self):
        """ Constructor """
        super().__init__()
        self._num_train_steps = 10000
        self._eval_on_train_data = False
        self._sample_1_of_n_eval_examples = None
        self._sample_1_of_n_eval_on_train_examples = 5
        self._checkpoint_dir = None
        self._eval_timeout = 3600
        self._use_tpu = False
        self._tpu_name = None
        self._num_workers = 1
        self._checkpoint_every_n = 1000
        self._record_summaries = True
    @property
    def num_train_steps(self):
        return self._flags.num_train_steps or self._num_train_steps
    @property
    def eval_on_train_data(self):
        return self._flags.eval_on_train_data or self._eval_on_train_data
    @property
    def sample_1_of_n_eval_examples(self):
        return self._flags.sample_1_of_n_eval_examples or self._sample_1_of_n_eval_examples
    @property
    def sample_1_of_n_eval_on_train_examples(self):
        return self._flags.sample_1_of_n_eval_on_train_examples or self._sample_1_of_n_eval_on_train_examples
    @property
    def checkpoint_dir(self):
        return self._flags.checkpoint_dir or self._checkpoint_dir
    @property
    def eval_timeout(self):
        return self._flags.eval_timeout or self._eval_timeout
    @property
    def use_tpu(self):
        return self._flags.use_tpu or self._use_tpu
    @property
    def tpu_name(self):
        return self._flags.tpu_name or self._tpu_name
    @property
    def num_workers(self):
        return self._flags.num_workers or self._num_workers
    @property
    def checkpoint_every_n(self):
        return self._flags.checkpoint_every_n or self._checkpoint_every_n
    @property
    def record_summaries(self):
        return self._flags.record_summaries or self._record_summaries

""" Arguments definition """
from absl import flags
if (not 'num_train_steps' in flags.FLAGS):
    flags.DEFINE_integer(
        'num_train_steps',
        10000,
        'Number of train steps.')
    flags.DEFINE_bool(
        'eval_on_train_data',
        False,
        'Enable evaluating on train data (only supported in distributed training).')
    flags.DEFINE_integer(
        'sample_1_of_n_eval_examples',
        None,
        'Will sample one of every n eval input examples, where n is provided.')
    flags.DEFINE_integer(
        'sample_1_of_n_eval_on_train_examples',
        5,
        'Will sample one of every n train input examples for evaluation, '
        'where n is provided. This is only used if `eval_training_data` is True.')
    flags.DEFINE_string(
        'checkpoint_dir',
        None,
        'Path to directory holding a checkpoint. If `checkpoint_dir` is provided, '
        'this binary operates in eval-only mode, writing resulting metrics to `model_dir`.')
    flags.DEFINE_integer(
        'eval_timeout',
        3600,
        'Number of seconds to wait for an evaluation checkpoint before exiting.')
    flags.DEFINE_bool(
        'use_tpu',
        False,
        'Whether the job is executing on a TPU.')
    flags.DEFINE_string(
        'tpu_name',
        None,
        'Name of the Cloud TPU for Cluster Resolvers.')
    flags.DEFINE_integer(
        'num_workers',
        1,
        'When num_workers > 1, training uses MultiWorkerMirroredStrategy. '
        'When num_workers = 1 it uses MirroredStrategy.')
    flags.DEFINE_integer(
        'checkpoint_every_n',
        1000,
        'Integer defining how often we checkpoint.')
    flags.DEFINE_boolean(
        'record_summaries',
        True,
        'Whether or not to record summaries during training.')

prm = TrainParameters()

print(f'Number of train steps: {prm.num_train_steps}')
print(f'Save a checkpoint every {prm.checkpoint_every_n} steps')
print('Train parameters configured')


#Utility functions
Some utility functions used for the steps

In [ ]:
import subprocess
import sys

def execute_subprocess(cmd):
    """
    Execute a subprocess returning each line of the standard output.
    Keyword arguments:
    cmd     -- the process to execute with its parameters
    """
    popen = subprocess.Popen(cmd, stdout=subprocess.PIPE, universal_newlines=True)
    for stdout_line in iter(popen.stdout.readline, ""):
        yield stdout_line 
    popen.stdout.close()
    return_code = popen.wait()
    if return_code:
        raise subprocess.CalledProcessError(return_code, cmd)

def execute(cmd):
    """
    Execute a subprocess printing its standard output.
    Keyword arguments:
    cmd     -- the process to execute with its parameters
    """
    for output in execute_subprocess(cmd):
        print(output, end="")

def execute_colab(fn):
    """
    Execute a function only in the Google Colab environment.
    Keyword arguments:
    fn      -- the function to execute
    """
    if ('google.colab' in sys.modules):
        fn()

def execute_non_colab(fn):
    """
    Execute a function only outside the Google Colab environment.
    Keyword arguments:
    fn      -- the function to execute
    """
    if (not 'google.colab' in sys.modules):
        fn()

if __name__ == '__main__':
    print('Utilities functions initialized')


#Object detection environment installation
This step installs a well known Python environment for the training

In [ ]:
import os
import datetime
from   pathlib import Path
import shutil
import sys
import tempfile

def install_object_detection():
    """
    Install a well known environment.
    """
    # Path of the python interpreter executable
    pythonPath = os.path.join(os.path.dirname(sys.executable), 'python3')
    if (not os.path.exists(pythonPath)):
        pythonPath = os.path.join(os.path.dirname(sys.executable), 'python')
    # Upgrade pip and setuptools
    execute([pythonPath, '-m', 'pip', 'install', '--upgrade', 'pip==21.0.1'])
    execute([pythonPath, '-m', 'pip', 'install', '--upgrade', 'setuptools==54.0.0'])
    # Install TensorFlow
    execute([pythonPath, '-m', 'pip', 'install', 'tensorflow==2.4.1'])
    # Install pygit2
    execute([pythonPath, '-m', 'pip', 'install', 'pygit2==1.5.0'])
    import pygit2
    # Progress class for the git output
    class GitCallbacks(pygit2.RemoteCallbacks):
        def __init__(self, credentials=None, certificate=None):
            self.dateTime = datetime.datetime.now()
            return super().__init__(credentials=credentials, certificate=certificate)
        def transfer_progress(self, stats):
            now = datetime.datetime.now()
            if ((now - self.dateTime).total_seconds() > 1):
                print('\rReceiving... Deltas [%d / %d], Objects [%d / %d]'%(stats.indexed_deltas, stats.total_deltas, stats.indexed_objects, stats.total_objects), end='', flush=True)
                self.dateTime = now
            if (stats.received_objects >= stats.total_objects and stats.indexed_objects >= stats.total_objects and stats.indexed_deltas >= stats.total_deltas):
                print('\r\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\rDone Deltas %d, Objects %d.'%(stats.total_objects, stats.total_objects))
            return super().transfer_progress(stats)
    # Directory of the TensorFlow object detection api
    odApiDir = os.path.join(tempfile.gettempdir(), 'tensorflow-object-detection-api')
    # Install the TensorFlow models
    if (not os.path.isdir(odApiDir)):
        # Create the callback for the progress
        callbacks = GitCallbacks();
        # Clone the TensorFlow models repository
        print('Cloning the TensorFlow models repository')
        pygit2.clone_repository('https://github.com/tensorflow/models.git', odApiDir, callbacks = callbacks)
        print('TensorFlow models repository cloned')
        # Checkout a well known commit
        repo = pygit2.Repository(odApiDir)
        ish = 'e356598a5b79a768942168b10d9c1acaa923bdb4'
        (commit, reference) = repo.resolve_refish(ish)
        repo.checkout_tree(commit)
        repo.reset(pygit2.Oid(hex=ish), pygit2.GIT_RESET_HARD)
        # Move to the research dir
        currentDir = os.getcwd()
        os.chdir(os.path.join(odApiDir, 'research'))
        # Install the protobuf tools
        execute([pythonPath, '-m', 'pip', 'install', 'grpcio-tools==1.32.0'])
        # Compile the protobufs
        import grpc_tools.protoc as protoc
        protoFiles = Path('object_detection/protos').rglob('*.proto')
        for protoFile in protoFiles:
            protoFilePath = str(protoFile)
            print('Compiling', protoFilePath)
            protoc.main(['grpc_tools.protoc', '--python_out=.', protoFilePath])
        # Install the object detection packages
        shutil.copy2('object_detection/packages/tf2/setup.py', '.')
        execute([pythonPath, '-m', 'pip', 'install', '.'])
        os.chdir(currentDir)
    sys.path.append(os.path.join(odApiDir, 'research'))
    sys.path.append(os.path.join(odApiDir, 'research/slim'))
    print('Installation completed.')

if __name__ == '__main__':
    install_object_detection()


# Environment initialization
In this section the environmento for the training will be initialized.

All necessary directories will be crated and the Google drive containing the images will be mounted. Follow the instruction for the mounting during the execution.

In [ ]:
from    absl import app
import  os
from    pathlib import Path
import  sys

def init_train_environment(prm = TrainParameters()):
    """
    Initialize the training environment with the right directories structure.
    Keyword arguments:
    prm     -- the train parameters
    """
    # Set the configuration for Google Colab
    if ('google.colab' in sys.modules):
        if (not os.path.exists('/mnt/MyDrive')):
            print('Mounting the GDrive')
            from google.colab import drive
            drive.mount('/mnt')
        # Check the existence of the train images dir
        gdriveOutputDir = os.path.join('/mnt', 'MyDrive', prm.train_images_dir)
        if (not os.path.isdir(gdriveOutputDir)):
            raise Exception('Error!!! The train images dir doesn`t exist')
        if (os.path.exists('/content/train-images')):
            os.unlink('/content/train-images')
        os.symlink(gdriveOutputDir, '/content/train-images', True)
        print(f"Google drive's {prm.train_images_dir} is linked to /content/train-images")
        prm.train_images_dir = '/content/train-images'
        # Check the existence of the evaluation images dir
        gdriveOutputDir = os.path.join('/mnt', 'MyDrive', prm.eval_images_dir)
        if (not os.path.isdir(gdriveOutputDir)):
            raise Exception('Error!!! The evaluation images dir doesn`t exist')
        if (os.path.exists('/content/eval-images')):
            os.unlink('/content/eval-images')
        os.symlink(gdriveOutputDir, '/content/eval-images', True)
        print(f"Google drive's {prm.eval_images_dir} is linked to /content/eval-images")
        prm.eval_images_dir = '/content/train-images'
        # Check the existence of the output directory
        gdriveOutputDir = os.path.join('/mnt', 'MyDrive', prm.model_dir)
        if (not os.path.isdir(gdriveOutputDir)):
            print('Creating the output directory')
            os.mkdir(gdriveOutputDir)
        if (os.path.exists('/content/trained-model')):
            os.unlink('/content/trained-model')
        os.symlink(gdriveOutputDir, '/content/trained-model', True)
        print(f"Google drive's {prm.model_dir} is linked to /content/trained-model")
        prm.model_dir = '/content/trained-model'
    else:
        if (not os.path.isdir(prm.train_images_dir)):
            raise Exception('Error!!! The train images dir doesn`t exist')
        print(f'Train images from {str(Path(prm.train_images_dir).resolve())}')
        if (not os.path.isdir(prm.eval_images_dir)):
            raise Exception('"Error!!! The evaluation images dir doesn`t exist')
        print(f'Train images from {str(Path(prm.eval_images_dir).resolve())}')
        if (not os.path.exists(prm.model_dir)):
            print('Creating the output dir')
            os.mkdir(prm.model_dir)
        print(f'The trained model will be in {str(Path(prm.model_dir).resolve())}')
    if (not os.path.exists(prm.annotations_dir)):
        os.mkdir(prm.annotations_dir)
    print(f'The annotations files will be in {str(Path(prm.annotations_dir).resolve())}')

if __name__ == '__main__':
    init_train_environment()


#Pre-trained model download
Dowloading the pre-trained model from the TensorFlow 2 model zoo.

In [ ]:
import  os
from    pathlib import Path
from    urllib import request

def download_pretrained_model(prm = BaseParameters()):
    """
    Download from the TensorFlow model zoo the pre-trained model.
    Keyword arguments:
    prm     -- the base parameters
    """
    if (not os.path.exists(prm.pre_trained_model_dir) or not os.path.exists(os.path.join(prm.pre_trained_model_dir, 'checkpoint', 'checkpoint'))):
        if (not os.path.exists(prm.pre_trained_model_base_dir)):
            os.mkdir(prm.pre_trained_model_base_dir)
        pre_trained_model_file = prm.pre_trained_model_dir + '.tar.gz'
        print(f'Downloading the pre-trained model {str(Path(pre_trained_model_file).name)}...')
        import tarfile
        request.urlretrieve(prm.model['DownloadPath'], pre_trained_model_file) # TODO: show progress
        print('Done.')
        print(f'Extracting the pre-trained model {str(Path(pre_trained_model_file).name)}...')
        tar = tarfile.open(pre_trained_model_file)
        tar.extractall(prm.pre_trained_model_base_dir)
        tar.close()
        os.remove(pre_trained_model_file)
        print(f'Pre-trained model is located at {str(Path(prm.pre_trained_model_dir).resolve())}')

if __name__ == '__main__':
    download_pretrained_model()


#TensorFlow's records
In this step there will be created the TensorFlow records from the annotated images and the file contained all the labels' indices.

In [ ]:
import  glob
import  io
import  os
from    pathlib import Path
import  shutil

class TFRecord:
    """Class for the TensorFlow records creation"""
    def __init__(self):
        """ Constructor """
        super().__init__()
        self._label_set = set()
        self._label_dict = dict()
    def class_text_to_int(self, row_label):
        """
        Convertion of the text of the labels to an integer index
        Keyword arguments:
        row_label   -- the label to convert to int
        """
        if (len(self._label_dict) == 0):
            count = len(self._label_set)
            labelIx = 1
            for label in self._label_set:
                self._label_dict[label] = labelIx
                labelIx += 1
        return self._label_dict[row_label]
    def create_tf_example(self, group, path):
        """
        TensorFlow example creator
        Keyword arguments:
        group   -- group's name
        path    -- path of the labeled images
        """
        from object_detection.utils import dataset_util
        from PIL import Image
        import tensorflow as tf
        with tf.compat.v1.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
            encoded_jpg = fid.read()
        encoded_jpg_io = io.BytesIO(encoded_jpg)
        image = Image.open(encoded_jpg_io)
        width, height = image.size
        filename = group.filename.encode('utf8')
        image_format = b'jpg'
        xmins = []
        xmaxs = []
        ymins = []
        ymaxs = []
        classes_text = []
        classes = []
        for index, row in group.object.iterrows():
            xmins.append(row['xmin'] / width)
            xmaxs.append(row['xmax'] / width)
            ymins.append(row['ymin'] / height)
            ymaxs.append(row['ymax'] / height)
            classes_text.append(row['class'].encode('utf8'))
            classes.append(self.class_text_to_int(row['class']))
        tf_example = tf.train.Example(features=tf.train.Features(feature={
            'image/height': dataset_util.int64_feature(height),
            'image/width': dataset_util.int64_feature(width),
            'image/filename': dataset_util.bytes_feature(filename),
            'image/source_id': dataset_util.bytes_feature(filename),
            'image/encoded': dataset_util.bytes_feature(encoded_jpg),
            'image/format': dataset_util.bytes_feature(image_format),
            'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
            'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
            'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
            'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
            'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
            'image/object/class/label': dataset_util.int64_list_feature(classes),
        }))
        return tf_example
    def create_tf_record(self, image_dir, output_file, labels_file = None, csv_file = None):
        """
        TensorFlow record creator
        Keyword arguments:
        image_dir   -- the directory containing the images
        output_file -- the output file path and name
        labels_file -- the optional output file path and name of the resulting labels file
        csv_file    -- the optional output file path and name of the csv file
        """
        import tensorflow as tf
        writer = tf.compat.v1.python_io.TFRecordWriter(output_file)
        path = os.path.join(image_dir)
        examples = self.xml_to_csv(image_dir)
        grouped = self.split(examples, 'filename')
        for group in grouped:
            tf_example = self.create_tf_example(group, path)
            writer.write(tf_example.SerializeToString())
        writer.close()
        print(f'Created the TFRecord file {str(Path(output_file).resolve())}')
        if labels_file is not None:
            from google.protobuf import text_format
            from object_detection.protos.string_int_label_map_pb2 import StringIntLabelMap, StringIntLabelMapItem
            msg = StringIntLabelMap()
            for id, name in enumerate(self._label_set, start = 1):
                msg.item.append(StringIntLabelMapItem(id = id, name = name))
            text = str(text_format.MessageToBytes(msg, as_utf8 = True), 'utf-8')
            with open(labels_file, 'w') as f:
                f.write(text)
            print(f'Created the labels map file {str(Path(labels_file).resolve())}')
        if csv_file is not None:
            examples.to_csv(csv_file, index = None)
            print(f'Created the CSV file {str(Path(csv_file).resolve())}')
    def split(self, df, group):
        """
        Split the labels in an image
        Keyword arguments:
        df      -- TensorFlow example
        group   -- group's name
        """
        from collections import namedtuple
        data = namedtuple('data', ['filename', 'object'])
        gb = df.groupby(group)
        return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]
    def xml_to_csv(self, path):
        """
        Convert the xml files generated by labeling image softwares into the cvs panda format
        Keyword arguments:
        path    -- Path of the generated csv file
        """
        import pandas as pd
        import xml.etree.ElementTree as ET
        xml_list = []
        for xml_file in glob.glob(path + '/*.xml'):
            tree = ET.parse(xml_file)
            root = tree.getroot()
            for member in root.findall('object'):
                value = (
                    root.find('filename').text,
                    int(root.find('size')[0].text),
                    int(root.find('size')[1].text),
                    member[0].text,
                    int(member[4][0].text),
                    int(member[4][1].text),
                    int(member[4][2].text),
                    int(member[4][3].text))
                xml_list.append(value)
                self._label_set.add(member[0].text)
        column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
        xml_df = pd.DataFrame(xml_list, columns = column_name)
        return xml_df

def create_tf_records(prm = BaseParameters()):
    """
    TensorFlow record files creator
    Keyword arguments:
    prm     -- Parameters
    """
    print("Creating TFRecord for the train images...")
    TFRecord().create_tf_record(
        prm.train_images_dir,
        os.path.join(prm.annotations_dir, 'train.record'),
        os.path.join(prm.annotations_dir, 'label_map.pbtxt'))
    print("Creating TFRecord for the evaluation images...")
    TFRecord().create_tf_record(
        prm.eval_images_dir,
        os.path.join(prm.annotations_dir, 'eval.record'))
    shutil.copy2(os.path.join(prm.annotations_dir, 'label_map.pbtxt'), prm.model_dir)
    print(f"The labels map file was copied to {(os.path.join(str(Path(prm.model_dir).resolve()), 'label_map.pbtxt'))}")

if __name__ == '__main__':
    create_tf_records()


#Train pipeline configuration
It configures the train pipeline using the original train pipeline of the pre-trained model but modifing some parameters as paths, number of labels, etc...

In [ ]:
import  os
import  shutil

def config_train_pipeline(prm = BaseParameters()):
    """
    Configure the training pipeline
    Keyword arguments:
    prm     -- Parameters
    """
    import tensorflow as tf
    from object_detection.protos import pipeline_pb2
    from google.protobuf import text_format
    # Copy the pipeline configuration file if it's not already present in the output dir
    print('Configuring the pipeline')
    output_file = os.path.join(prm.annotations_dir, 'pipeline.config')
    shutil.copy2(os.path.join(prm.pre_trained_model_dir, 'pipeline.config'), output_file)
    # Configuring the pipeline
    pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
    with tf.io.gfile.GFile(output_file, "r") as f:
        proto_str = f.read()
        text_format.Merge(proto_str, pipeline_config)
    pipeline_config.model.ssd.num_classes = 1 # TODO define
    pipeline_config.model.ssd.image_resizer.fixed_shape_resizer.height = prm.model['height']
    pipeline_config.model.ssd.image_resizer.fixed_shape_resizer.width = prm.model['width']
    pipeline_config.train_config.batch_size = prm.model['batch_size']
    pipeline_config.train_config.fine_tune_checkpoint = os.path.join(prm.pre_trained_model_dir, 'checkpoint', 'ckpt-0')
    pipeline_config.train_config.fine_tune_checkpoint_type = 'detection'
    pipeline_config.train_input_reader.label_map_path = os.path.join(prm.annotations_dir, 'label_map.pbtxt')
    pipeline_config.train_input_reader.tf_record_input_reader.input_path[0] = os.path.join(prm.annotations_dir, 'train.record')
    pipeline_config.eval_input_reader[0].label_map_path = os.path.join(prm.annotations_dir, 'label_map.pbtxt')
    pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[0] = os.path.join(prm.annotations_dir, 'eval.record')
    config_text = text_format.MessageToString(pipeline_config)
    with tf.io.gfile.GFile(output_file, 'wb') as f:
        f.write(config_text)
    shutil.copy2(output_file, prm.model_dir)
    print('The train pipeline content is:')
    print(str(config_text))

if __name__ == '__main__':
    config_train_pipeline()


#Train loop
The main train loop. It trains the model and put it in the output directory.

It can be stopped before the complete train when a considerable result is reached.

In [ ]:
import  os

def main(_):
    """
    Train loop function
    """
    import tensorflow as tf
    from object_detection import model_lib_v2
    if prm.checkpoint_dir:
        model_lib_v2.eval_continuously(
            pipeline_config_path = os.path.join(prm.annotations_dir, 'pipeline.config'),
            model_dir = prm.num_train_steps,
            sample_1_of_n_eval_examples = prm.sample_1_of_n_eval_examples,
            sample_1_of_n_eval_on_train_examples = prm.sample_1_of_n_eval_on_train_examples,
            checkpoint_dir = prm.checkpoint_dir,
            wait_interval = 300,
            timeout = prm.eval_timeout)
    else:
        if prm.use_tpu:
            # TPU is automatically inferred if tpu_name is None and
            # we are running under cloud ai-platform.
            resolver = tf.distribute.cluster_resolver.TPUClusterResolver(prm.tpu_name)
            tf.config.experimental_connect_to_cluster(resolver)
            tf.tpu.experimental.initialize_tpu_system(resolver)
            strategy = tf.distribute.experimental.TPUStrategy(resolver)
        elif prm.num_workers > 1:
            strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()
        else:
            strategy = tf.compat.v2.distribute.MirroredStrategy()
    with strategy.scope():
        print(prm.annotations_dir)
        print(prm.model_dir)
        print(prm.num_train_steps)
        print(prm.use_tpu)
        print(prm.checkpoint_every_n)
        print(prm.record_summaries)
        model_lib_v2.train_loop(
            pipeline_config_path = os.path.join(prm.annotations_dir, 'pipeline.config'),
            model_dir = prm.model_dir,
            train_steps = prm.num_train_steps,
            use_tpu = prm.use_tpu,
            checkpoint_every_n = prm.checkpoint_every_n,
            record_summaries = prm.record_summaries)

if __name__ == '__main__':
    import tensorflow as tf
    tf.compat.v1.app.run()
